In [2]:
import requests
import pandas as pd
import json
import time
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
import threading
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import re
from tkinter import filedialog
import configparser
import os
import concurrent.futures
import queue
import webbrowser  # 이 코드를 다른 import문 근처에 추가
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import openpyxl
from openpyxl.styles import Font, PatternFill, Alignment

class ComplexSelectionDialog:
    def __init__(self, parent, title, complex_list):
        self.result = None
        
        # 대화상자 생성
        self.dialog = tk.Toplevel(parent)
        self.dialog.title(title)
        self.dialog.geometry("700x500")  # 크기를 더 크게 설정 (원래 600x400)
        self.dialog.minsize(700, 500)    # 최소 크기 설정
        self.dialog.transient(parent)
        self.dialog.grab_set()
    
        # 프레임 생성
        frame = ttk.Frame(self.dialog, padding="10")
        frame.pack(fill=tk.BOTH, expand=True)
        
        # 라벨 생성
        ttk.Label(frame, text="검색된 단지 목록. 선택하세요:").pack(anchor="w", pady=(0, 5))
        
        # 트리뷰 생성
        columns = ("name", "address")
        self.tree = ttk.Treeview(frame, columns=columns, show="headings", height=15)
        self.tree.heading("name", text="단지명")
        self.tree.heading("address", text="주소")
        
        self.tree.column("name", width=250)
        self.tree.column("address", width=400)
        
        # 스크롤바 생성
        scrollbar = ttk.Scrollbar(frame, orient="vertical", command=self.tree.yview)
        self.tree.configure(yscrollcommand=scrollbar.set)
        
        self.tree.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        
        # 데이터 추가
        for item in complex_list:
            self.tree.insert("", "end", values=(item["name"], item["address"]), tags=(item["id"],))
        
        # 버튼 프레임 - 패딩 늘림
        button_frame = ttk.Frame(self.dialog, padding="20")  # 패딩 값 증가 (원래 10)
        button_frame.pack(fill="x", pady=10)  # 상하 여백 추가
        
        # 확인 버튼
        select_button = ttk.Button(button_frame, text="선택", command=self.on_select, width=10)  # 버튼 너비 명시
        select_button.pack(side="right", padx=10)  # 여백 증가
        
        # 취소 버튼
        cancel_button = ttk.Button(button_frame, text="취소", command=self.on_cancel, width=10)  # 버튼 너비 명시
        cancel_button.pack(side="right", padx=10)  # 여백 증가
        
        # 더블 클릭 이벤트 바인딩
        self.tree.bind("<Double-1>", lambda e: self.on_select())
        
        # 창 중앙 배치
        self.center_dialog()
        
        # 대화상자가 닫힐 때까지 대기
        parent.wait_window(self.dialog)
    
    def center_dialog(self):
        """대화상자를 화면 중앙에 배치"""
        self.dialog.update_idletasks()
        
        # 화면 크기 가져오기
        screen_width = self.dialog.winfo_screenwidth()
        screen_height = self.dialog.winfo_screenheight()
        
        # 창 크기 가져오기
        dialog_width = self.dialog.winfo_width()
        dialog_height = self.dialog.winfo_height()
        
        # 화면 중앙 위치 계산
        x = (screen_width - dialog_width) // 2
        y = (screen_height - dialog_height) // 2
        
        # 위치 설정
        self.dialog.geometry(f"+{x}+{y}")
    
    def on_select(self):
        selected_items = self.tree.selection()
        if selected_items:
            item_id = selected_items[0]
            item_values = self.tree.item(item_id)['values']
            item_tags = self.tree.item(item_id)['tags']
            
            self.result = {
                "name": item_values[0],
                "address": item_values[1],
                "id": item_tags[0]
            }
            
        self.dialog.destroy()
    
    def on_cancel(self):
        self.dialog.destroy()

class NaverRealEstateApp:
    def __init__(self, root):
        self.root = root
        self.root.title("네이버 부동산 매물 수집기")
        self.root.geometry("500x220")  # 창 크기 약간 늘림
        self.root.resizable(True, True)
        self.log_queue = queue.Queue()
        self.root.after(100, self.process_log_queue)
        # 변수 초기화
        self.complex_data = None
        self.driver = None
        # 설정 관리
        self.config = configparser.ConfigParser()
        self.save_path = os.path.expanduser("~/Documents")  # 기본 저장 경로
        self.load_config()
        
        # 필요한 패키지 확인
        self.check_required_packages()
        
        # UI 구성
        self.setup_ui()

    def process_log_queue(self):
        """로그 큐의 메시지를 처리 (최소한의 상태 표시만)"""
        try:
            while True:
                message = self.log_queue.get_nowait()
                
                # 콘솔에만 출력 (디버깅용)
                print(message)
                
                # 매우 간단한 상태 표시
                if "단지 검색을 시작합니다" in message:
                    self.status_label.config(text="검색 중...")
                elif "매물 정보 수집을 시작합니다" in message:
                    self.status_label.config(text="매물 수집 중...")
                elif "총 " in message and "개의 매물 정보가 저장되었습니다" in message:
                    match = re.search(r'총 (\d+)개', message)
                    if match:
                        count = match.group(1)
                        self.status_label.config(text=f"{count}건 매물 검색 완료")
                elif "엑셀 파일" in message and "생성 완료" in message:
                    self.status_label.config(text="파일 저장 완료")
                
                self.log_queue.task_done()
        except queue.Empty:
            pass
        except Exception as e:
            # 오류 방지를 위한 예외 처리
            print(f"로그 처리 중 오류: {str(e)}")
        finally:
            # 100ms 후에 다시 호출
            self.root.after(100, self.process_log_queue)


            
    def setup_ui(self):
        # 프레임 구성
        search_frame = ttk.LabelFrame(self.root, text="아파트 단지 검색")
        search_frame.pack(fill="x", padx=10, pady=5)
        
        # 검색 프레임 구성
        ttk.Label(search_frame, text="단지명:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        self.search_entry = ttk.Entry(search_frame, width=40)
        self.search_entry.grid(row=0, column=1, padx=5, pady=5, sticky="ew")
        self.search_entry.bind("<Return>", lambda event: self.start_search())
        
        self.search_button = ttk.Button(search_frame, text="검색 및 매물 수집", command=self.start_search)
        self.search_button.grid(row=0, column=2, padx=5, pady=5)
        
        # 설정 버튼 추가
        self.settings_button = ttk.Button(search_frame, text="⚙", width=3, command=self.open_settings)
        self.settings_button.grid(row=0, column=3, padx=5, pady=5)
        
        search_frame.columnconfigure(1, weight=1)
        
        # 박스그래프 옵션 프레임 추가
        box_plot_frame = ttk.Frame(self.root)
        box_plot_frame.pack(fill="x", padx=10, pady=5)
        
        # 박스그래프 체크박스 추가
        self.box_plot_var = tk.BooleanVar(value=False)
        self.box_plot_checkbox = ttk.Checkbutton(
            box_plot_frame, 
            text="박스그래프 그리기", 
            variable=self.box_plot_var,
            command=self.toggle_box_plot_input
        )
        self.box_plot_checkbox.pack(side="left", padx=5)
        
        # 전용면적 입력 필드 추가 (초기에는 비활성화)
        ttk.Label(box_plot_frame, text="전용면적(㎡):").pack(side="left", padx=5)
        self.area_entry = ttk.Entry(box_plot_frame, width=10, state="disabled")
        self.area_entry.pack(side="left", padx=5)
        
        # 상태 표시 레이블 - 중앙에 크게 표시
        self.status_label = tk.Label(
            self.root, 
            text="단지명을 입력하고 검색하세요", 
            font=("맑은 고딕", 9),
            pady=15
        )
        self.status_label.pack(fill="x")
        
        # 제작자 정보 프레임
        author_frame = ttk.Frame(self.root)
        author_frame.pack(side="bottom", fill="x", padx=10, pady=10)
        
        # 제작자 이름 (클릭 가능한 링크)
        author_label = tk.Label(author_frame, text="만든이 부태리", font=("맑은 고딕", 9, "bold"), fg="black", cursor="hand2")
        author_label.pack(anchor="center")
        author_label.bind("<Button-1>", lambda e: self.open_blog())
        
        # 블로그 주소
        blog_label = tk.Label(author_frame, text="https://blog.naver.com/landlover333", font=("맑은 고딕", 8))
        blog_label.pack(anchor="center")
        blog_label.bind("<Button-1>", lambda e: self.open_blog())

    def toggle_box_plot_input(self):
        """박스그래프 체크박스 상태에 따라 전용면적 입력 필드 활성화/비활성화"""
        if self.box_plot_var.get():
            self.area_entry.config(state="normal")
        else:
            self.area_entry.config(state="disabled")
    
    def create_box_plot(self, df, area):
        """특정 전용면적에 대한 매매가와 전세 보증금의 박스그래프 생성"""
        try:
            # 데이터프레임 열 확인 로깅
            self.log(f"데이터프레임 열: {df.columns.tolist()}")
            
            # 전용면적 열이 있는지 확인
            if '전용면적' not in df.columns:
                self.log("'전용면적' 열을 찾을 수 없습니다.")
                messagebox.showwarning("경고", "'전용면적' 열을 찾을 수 없습니다. 데이터 구조를 확인해주세요.")
                return
                
            # 거래유형 열이 있는지 확인
            if '거래유형' not in df.columns:
                self.log("'거래유형' 열을 찾을 수 없습니다.")
                messagebox.showwarning("경고", "'거래유형' 열을 찾을 수 없습니다. 데이터 구조를 확인해주세요.")
                return
            
            # 층 정보 열이 있는지 확인
            floor_column = '층/전체층'
            if floor_column not in df.columns:
                self.log(f"'{floor_column}' 열을 찾을 수 없습니다.")
                messagebox.showwarning("경고", f"'{floor_column}' 열을 찾을 수 없습니다. 데이터 구조를 확인해주세요.")
                return
            
            # 단지명 열이 있는지 확인 및 단지명 추출
            complex_name = ""
            if '단지명' in df.columns and not df['단지명'].empty:
                # 첫 번째 단지명 사용 (모든 행의 단지명이 같다고 가정)
                complex_name = df['단지명'].iloc[0]
                self.log(f"단지명: {complex_name}")
            
            # 전용면적 값 로깅 (처음 10개)
            sample_areas = df['전용면적'].head(10).tolist()
            self.log(f"전용면적 샘플 값 (처음 10개): {sample_areas}")
            
            # 거래유형 값 로깅 (고유 값 모두)
            unique_types = df['거래유형'].unique().tolist()
            self.log(f"거래유형 고유 값: {unique_types}")
            
            # 소수점 버림 처리하여 정수로 변환 (NaN 및 문자열 처리 강화)
            df['전용면적_정수'] = df['전용면적'].apply(
                lambda x: int(float(x)) if pd.notnull(x) and str(x).replace('.', '', 1).isdigit() else None
            )
            
            # 전용면적_정수 값 로깅 (처음 10개)
            sample_int_areas = df['전용면적_정수'].head(10).tolist()
            self.log(f"전용면적_정수 샘플 값 (처음 10개): {sample_int_areas}")
            
            # 층 정보 처리 함수
            def extract_floor(floor_info):
                if pd.isna(floor_info) or not floor_info:
                    return None
                
                floor_str = str(floor_info).lower()
                
                # '중', '고' 포함 처리 (층 제외)
                if '중' in floor_str or '고' in floor_str:
                    return 5  # 5층 이상으로 간주
                
                # 숫자만 추출 ('5/10' -> 5)
                try:
                    floor_num = floor_str.split('/')[0].strip()
                    # 숫자가 아닌 문자 제거
                    floor_num = ''.join(c for c in floor_num if c.isdigit())
                    if floor_num:
                        return int(floor_num)
                except:
                    pass
                
                return None
            
            # 층 정보 추출
            df['층수'] = df[floor_column].apply(extract_floor)
            
            # 층 정보 로깅 (처음 10개)
            sample_floors = list(zip(df[floor_column].head(10).tolist(), df['층수'].head(10).tolist()))
            self.log(f"층 정보 샘플 값 (원본, 추출) (처음 10개): {sample_floors}")
            
            # 대상 전용면적의 매물 수 로깅
            matching_count = df[df['전용면적_정수'] == area].shape[0]
            self.log(f"전용면적 {area}㎡에 해당하는 매물 수: {matching_count}")
            
            # 주어진 전용면적과 일치하는 매물 필터링
            filtered_df = df[df['전용면적_정수'] == area]
            
            if filtered_df.empty:
                self.log(f"전용면적 {area}㎡에 해당하는 매물이 없습니다.")
                messagebox.showinfo("알림", f"전용면적 {area}㎡에 해당하는 매물이 없습니다.")
                return
                
            # 데이터 준비 - 거래유형이 'A1'(매매)인 매물의 매매가 데이터
            deal_df = filtered_df[filtered_df['거래유형'].isin(['A1', '매매'])]
            
            # '매매가' 열이 있는지 확인
            if '매매가' not in df.columns:
                self.log("'매매가' 열을 찾을 수 없습니다.")
                messagebox.showwarning("경고", "'매매가' 열을 찾을 수 없습니다. 데이터 구조를 확인해주세요.")
                return
            
            deal_data = deal_df[deal_df['매매가'] > 0]['매매가'].dropna().tolist()
            self.log(f"매매(A1) 거래유형 매물 수: {len(deal_df)}, 유효한 매매가 데이터 수: {len(deal_data)}")
            
            # 5층 이상 매매 매물 찾기
            high_floor_deal_df = deal_df[(deal_df['층수'] >= 5) & (deal_df['매매가'] > 0)]
            high_floor_deal_min = None
            if not high_floor_deal_df.empty:
                high_floor_deal_min = high_floor_deal_df['매매가'].min()
                self.log(f"5층 이상 매매 매물 수: {len(high_floor_deal_df)}, 최저가: {high_floor_deal_min}")
            
            # 데이터 준비 - 거래유형이 'B1'(전세)인 매물의 보증금 데이터
            deposit_df = filtered_df[filtered_df['거래유형'].isin(['B1', '전세'])]
            
            # '보증금' 열이 있는지 확인
            if '보증금' not in df.columns:
                self.log("'보증금' 열을 찾을 수 없습니다.")
                messagebox.showwarning("경고", "'보증금' 열을 찾을 수 없습니다. 데이터 구조를 확인해주세요.")
                return
            
            deposit_data = deposit_df[deposit_df['보증금'] > 0]['보증금'].dropna().tolist()
            self.log(f"전세(B1) 거래유형 매물 수: {len(deposit_df)}, 유효한 보증금 데이터 수: {len(deposit_data)}")
            
            # 5층 이상 전세 매물 찾기
            high_floor_deposit_df = deposit_df[(deposit_df['층수'] >= 5) & (deposit_df['보증금'] > 0)]
            high_floor_deposit_min = None
            if not high_floor_deposit_df.empty:
                high_floor_deposit_min = high_floor_deposit_df['보증금'].min()
                self.log(f"5층 이상 전세 매물 수: {len(high_floor_deposit_df)}, 최저가: {high_floor_deposit_min}")
            
            if not deal_data and not deposit_data:
                self.log(f"전용면적 {area}㎡에 해당하는 매매가/전세 보증금 데이터가 없습니다.")
                messagebox.showinfo("알림", f"전용면적 {area}㎡에 해당하는 매매가/전세 보증금 데이터가 없습니다.")
                return
                
            # matplotlib 사용하여 박스그래프 생성
            import matplotlib
            matplotlib.use('Agg')  # 백엔드를 Agg로 설정 (화면에 직접 출력하지 않음)
            
            import matplotlib.pyplot as plt
            from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
            import numpy as np
            import matplotlib.ticker as ticker
            
            # 이미 열린 matplotlib 창 닫기
            plt.close('all')
            
            # 숫자 표시 형식 지정 함수
            def format_number(x, pos):
                return '{:,}'.format(int(x))
            
            # 한글 폰트 설정
            import matplotlib.font_manager as fm
            
            # 한글 폰트 설정 시도
            try:
                font_path = 'C:/Windows/Fonts/malgun.ttf'  # Windows 기본 맑은 고딕 폰트
                font_prop = fm.FontProperties(fname=font_path)
                plt.rc('font', family=font_prop.get_name())
            except:
                self.log("기본 한글 폰트를 찾을 수 없습니다. 시스템 기본 폰트를 사용합니다.")
                plt.rc('font', family='sans-serif')
            
            plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지
            
            # 창 제목에 단지명 추가
            window_title = f"{complex_name} - 전용면적 {area}㎡ 매물 가격 분석" if complex_name else f"전용면적 {area}㎡ 매물 가격 분석"
            
            # 이 부분이 중요합니다: 기존 창이 있으면 먼저 닫습니다
            for widget in self.root.winfo_children():
                if isinstance(widget, tk.Toplevel) and '매물 가격 분석' in widget.title():
                    widget.destroy()
            
            # 새 창 생성 - 창 크기 키우기
            plot_window = tk.Toplevel(self.root)
            plot_window.title(window_title)
            plot_window.geometry("1000x800")  # 창 크기 더 키움
            
            # Notebook 위젯 생성 (탭 컨테이너)
            notebook = ttk.Notebook(plot_window)
            notebook.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
            
            # 매매가 그래프와 통계 탭
            if deal_data:
                deal_frame = ttk.Frame(notebook)
                notebook.add(deal_frame, text="매매가 분석")
                
                # 매매가 통계 프레임
                deal_stats_frame = ttk.LabelFrame(deal_frame, text="매매가 통계 정보")
                deal_stats_frame.pack(fill="x", padx=10, pady=5)
                
                # 통계 정보 계산
                deal_avg = np.mean(deal_data)
                deal_median = np.median(deal_data)
                deal_min = np.min(deal_data)
                deal_max = np.max(deal_data)
                deal_std = np.std(deal_data)
                
                # 통계 정보 텍스트 생성 (천 단위 구분자 사용, 소수점 없음, "만원" 제외)
                deal_stats_text = f"매매가 통계:\n"
                deal_stats_text += f"  매물 수: {len(deal_data)}개\n"
                deal_stats_text += f"  평균: {int(deal_avg):,}\n"
                deal_stats_text += f"  중앙값: {int(deal_median):,}\n"
                deal_stats_text += f"  최소값: {int(deal_min):,}\n"
                deal_stats_text += f"  최대값: {int(deal_max):,}\n"
                deal_stats_text += f"  표준편차: {int(deal_std):,}\n"
                
                # 5층 이상 최저가 추가
                if high_floor_deal_min is not None:
                    deal_stats_text += f"  저층(4층 이하) 제외 최저가: {int(high_floor_deal_min):,}\n"
                
                # 통계 정보 표시
                deal_stats_label = ttk.Label(deal_stats_frame, text=deal_stats_text, justify="left")
                deal_stats_label.pack(padx=10, pady=10)
                
                # 매매가 그래프 프레임
                deal_graph_frame = ttk.Frame(deal_frame)
                deal_graph_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=5)
                
                # 매매가 그래프 생성 - 그래프 크기 키우기
                deal_fig = plt.figure(figsize=(10, 6))
                deal_ax = deal_fig.add_subplot(111)
                deal_box = deal_ax.boxplot(deal_data, patch_artist=True)
                deal_box['boxes'][0].set_facecolor('lightblue')
                
                # 그래프 제목 및 레이블 설정 (단위 표시 제외)
                graph_title = f'{complex_name} - 전용면적 {area}㎡ 매매가 분포' if complex_name else f'전용면적 {area}㎡ 매매가 분포'
                deal_ax.set_title(graph_title, fontsize=14)
                deal_ax.set_ylabel('매매가', fontsize=12)
                deal_ax.set_xticklabels([f'매매가\n(n={len(deal_data)})'])
                
                # y축 포맷 설정 (천 단위 구분자)
                deal_ax.yaxis.set_major_formatter(ticker.FuncFormatter(format_number))
                
                # 그리드 추가
                deal_ax.grid(True, linestyle='--', alpha=0.7)
                
                # 여백 조정으로 그래프가 잘리지 않게 함
                deal_fig.tight_layout()
                
                # 그래프를 Tkinter 창에 표시
                deal_canvas = FigureCanvasTkAgg(deal_fig, master=deal_graph_frame)
                deal_canvas.draw()
                deal_canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
            
            # 전세 보증금 그래프와 통계 탭
            if deposit_data:
                deposit_frame = ttk.Frame(notebook)
                notebook.add(deposit_frame, text="전세 보증금 분석")
                
                # 전세 보증금 통계 프레임
                deposit_stats_frame = ttk.LabelFrame(deposit_frame, text="전세 보증금 통계 정보")
                deposit_stats_frame.pack(fill="x", padx=10, pady=5)
                
                # 통계 정보 계산
                deposit_avg = np.mean(deposit_data)
                deposit_median = np.median(deposit_data)
                deposit_min = np.min(deposit_data)
                deposit_max = np.max(deposit_data)
                deposit_std = np.std(deposit_data)
                
                # 통계 정보 텍스트 생성 (천 단위 구분자 사용, 소수점 없음, "만원" 제외)
                deposit_stats_text = f"전세 보증금 통계:\n"
                deposit_stats_text += f"  매물 수: {len(deposit_data)}개\n"
                deposit_stats_text += f"  평균: {int(deposit_avg):,}\n"
                deposit_stats_text += f"  중앙값: {int(deposit_median):,}\n"
                deposit_stats_text += f"  최소값: {int(deposit_min):,}\n"
                deposit_stats_text += f"  최대값: {int(deposit_max):,}\n"
                deposit_stats_text += f"  표준편차: {int(deposit_std):,}\n"
                
                # 5층 이상 최저가 추가
                if high_floor_deposit_min is not None:
                    deposit_stats_text += f"  저층(4층 이하) 제외 최저가: {int(high_floor_deposit_min):,}\n"
                
                # 통계 정보 표시
                deposit_stats_label = ttk.Label(deposit_stats_frame, text=deposit_stats_text, justify="left")
                deposit_stats_label.pack(padx=10, pady=10)
                
                # 전세 보증금 그래프 프레임
                deposit_graph_frame = ttk.Frame(deposit_frame)
                deposit_graph_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=5)
                
                # 전세 보증금 그래프 생성 - 그래프 크기 키우기
                deposit_fig = plt.figure(figsize=(10, 6))
                deposit_ax = deposit_fig.add_subplot(111)
                deposit_box = deposit_ax.boxplot(deposit_data, patch_artist=True)
                deposit_box['boxes'][0].set_facecolor('lightgreen')
                
                # 그래프 제목 및 레이블 설정 (단위 표시 제외)
                graph_title = f'{complex_name} - 전용면적 {area}㎡ 전세 보증금 분포' if complex_name else f'전용면적 {area}㎡ 전세 보증금 분포'
                deposit_ax.set_title(graph_title, fontsize=14)
                deposit_ax.set_ylabel('전세 보증금', fontsize=12)
                deposit_ax.set_xticklabels([f'전세 보증금\n(n={len(deposit_data)})'])
                
                # y축 포맷 설정 (천 단위 구분자)
                deposit_ax.yaxis.set_major_formatter(ticker.FuncFormatter(format_number))
                
                # 그리드 추가
                deposit_ax.grid(True, linestyle='--', alpha=0.7)
                
                # 여백 조정으로 그래프가 잘리지 않게 함
                deposit_fig.tight_layout()
                
                # 그래프를 Tkinter 창에 표시
                deposit_canvas = FigureCanvasTkAgg(deposit_fig, master=deposit_graph_frame)
                deposit_canvas.draw()
                deposit_canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
            
            # 요약 탭 (매매와 전세 비교 텍스트) - 중앙값 사용
            if deal_data and deposit_data:
                summary_frame = ttk.Frame(notebook)
                notebook.add(summary_frame, text="요약 비교")
                
                # 요약 제목에 단지명 추가
                summary_title = f"{complex_name} - 전용면적 {area}㎡ 매물 가격 요약" if complex_name else f"전용면적 {area}㎡ 매물 가격 요약"
                summary_label = ttk.Label(
                    summary_frame, 
                    text=summary_title, 
                    font=("맑은 고딕", 12, "bold")
                )
                summary_label.pack(pady=10)
                
                # 비교 통계 생성
                comparison_frame = ttk.LabelFrame(summary_frame, text="매매가와 전세 보증금 비교")
                comparison_frame.pack(fill="x", padx=20, pady=10)
                
                # 비교 텍스트 생성 (중앙값 사용, 천 단위 구분자 사용, 소수점 없음)
                comparison_text = f"매매가 중앙값: {int(deal_median):,} (n={len(deal_data)})\n"
                comparison_text += f"전세 보증금 중앙값: {int(deposit_median):,} (n={len(deposit_data)})\n\n"
                
                # 전세가율 계산 (전세 중앙값 / 매매 중앙값 * 100)
                ratio = (deposit_median / deal_median) * 100
                comparison_text += f"전세가율 (전세가/매매가): {int(ratio)}%\n\n"
                
                # 5층 이상 최저가 정보 추가
                if high_floor_deal_min is not None:
                    comparison_text += f"매매 저층(4층 이하) 제외 최저가: {int(high_floor_deal_min):,}\n"
                
                if high_floor_deposit_min is not None:
                    comparison_text += f"전세 저층(4층 이하) 제외 최저가: {int(high_floor_deposit_min):,}\n"
                
                comparison_label = ttk.Label(comparison_frame, text=comparison_text, justify="left")
                comparison_label.pack(padx=10, pady=10)
            
            # 창 중앙에 배치
            plot_window.update_idletasks()
            screen_width = plot_window.winfo_screenwidth()
            screen_height = plot_window.winfo_screenheight()
            window_width = plot_window.winfo_width()
            window_height = plot_window.winfo_height()
            x = (screen_width - window_width) // 2
            y = (screen_height - window_height) // 2
            plot_window.geometry(f"+{x}+{y}")
            
            # 창을 최상위로 설정하여 포커스 받도록 함
            plot_window.lift()
            plot_window.focus_force()
            
            # 창이 닫힐 때 그래프 클린업
            def on_closing():
                plt.close('all')
                plot_window.destroy()
            
            plot_window.protocol("WM_DELETE_WINDOW", on_closing)
            
        except Exception as e:
            self.log(f"박스그래프 생성 중 오류 발생: {str(e)}")
            import traceback
            self.log(traceback.format_exc())
            messagebox.showerror("오류", f"박스그래프 생성 중 오류가 발생했습니다.\n{str(e)}")
    def open_blog(self):
        """블로그 링크 열기"""
        webbrowser.open("https://blog.naver.com/landlover333")        
        
    def load_config(self):
        """설정 파일 로드"""
        try:
            # 설정 파일 경로
            config_path = os.path.join(os.path.expanduser("~"), ".naver_realestate_config.ini")
            
            if os.path.exists(config_path):
                self.config.read(config_path)
                if 'Settings' in self.config and 'save_path' in self.config['Settings']:
                    path = self.config['Settings']['save_path']
                    if os.path.exists(path):
                        self.save_path = path
        except Exception as e:
            print(f"설정 로드 오류: {str(e)}")
        
    def save_config(self):
        """설정 파일 저장"""
        try:
            # 설정 파일 경로
            config_path = os.path.join(os.path.expanduser("~"), ".naver_realestate_config.ini")
            
            if 'Settings' not in self.config:
                self.config['Settings'] = {}
            
            self.config['Settings']['save_path'] = self.save_path
            
            with open(config_path, 'w') as f:
                self.config.write(f)
        except Exception as e:
            print(f"설정 저장 오류: {str(e)}")
        
    def open_settings(self):
        """설정 창 열기"""
        # 폴더 선택 대화상자
        path = filedialog.askdirectory(initialdir=self.save_path, title="저장할 폴더 선택")
        
        if path:
            self.save_path = path
            self.save_config()
            self.log(f"저장 경로가 변경되었습니다: {path}")

    
    # log 메서드를 스레드 안전하게 변경
    def log(self, message):
        """스레드 안전한 로깅"""
        # 로그 큐에 메시지 추가
        self.log_queue.put(message)
        # 나머지 코드 제거

    
    def start_search(self):
        """단지명으로 검색 시작 (스레드로 실행)"""
        search_keyword = self.search_entry.get().strip()
        if not search_keyword:
            messagebox.showwarning("경고", "검색할 단지명을 입력하세요.")
            return
        
        self.search_button.config(state=tk.DISABLED)
        threading.Thread(target=self.search_complex, args=(search_keyword,), daemon=True).start()
    
    def search_complex(self, search_keyword):
        """단지명으로 검색 및 단지번호 찾기"""
        try:
            self.log(f"'{search_keyword}' 단지 검색을 시작합니다...")
            
            # Chrome 옵션 설정
            chrome_options = Options()
            chrome_options.add_argument("--headless")  # 헤드리스 모드 (화면 표시 없음)
            chrome_options.add_argument("--window-size=1920,1080")
            chrome_options.add_argument("--disable-gpu")
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")
            chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36")
            
            self.log("웹 브라우저를 초기화하는 중...")

            # driver 초기화 부분을 다음과 같이 수정 (search_complex 메서드와 search_complex_detail 메서드 모두)
            # 기존 코드:
            # self.driver = webdriver.Chrome(options=chrome_options)
            
            # 수정할 코드:
            service = Service(ChromeDriverManager().install())
            self.driver = webdriver.Chrome(service=service, options=chrome_options)
            
            # 네이버 부동산 접속
            self.log("네이버 부동산에 접속 중...")
            self.driver.get("https://fin.land.naver.com/")
            time.sleep(2)
            
            # 검색 버튼 클릭
            self.log("검색 버튼 클릭 중...")
            search_button = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "svg[viewBox='0 0 24 24']"))
            )
            search_button.click()
            time.sleep(1)
            
            # 검색창에 단지명 입력
            self.log(f"검색창에 '{search_keyword}' 입력 중...")
            search_input = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.ID, "query"))
            )
            search_input.clear()
            search_input.send_keys(search_keyword)
            search_input.send_keys(Keys.ENTER)
            time.sleep(3)
            
            # 검색 결과 확인
            complex_list = []

            # 페이지 소스 확인 (디버깅용)
            self.log("현재 페이지 HTML 구조 확인 중...")
            page_source = self.driver.page_source
            if "keyword_auto" in page_source:
                self.log("자동완성 결과 영역을 감지했습니다.")
            if "_keywordAutoCompleteUl" in page_source:
                self.log("자동완성 리스트를 감지했습니다.")
            
            # 자동완성 영역 찾기 시도
            try:
                auto_complete = self.driver.find_element(By.ID, "keyword_auto")
                self.log(f"자동완성 영역 표시 상태: {auto_complete.get_attribute('style')}")
            except:
                self.log("자동완성 영역을 찾을 수 없습니다.")
            
      
           
            try:
                # 기존 자동완성 코드 모두 제거 후, 아래 코드로 대체
                
                # 검색 결과 목록 찾기
                self.log("검색 결과 목록을 찾는 중...")
                time.sleep(2)  # 검색 결과가 로드될 시간 확보
                
                # 복합 단지 목록 찾기
                complex_items = self.driver.find_elements(By.CSS_SELECTOR, "#complex_list_ul .result_item")
                
                if complex_items and len(complex_items) > 0:
                    self.log(f"{len(complex_items)}개의 단지 검색 결과를 찾았습니다.")
                    
                    # 검색 결과 목록에서 단지 정보 추출
                    for item in complex_items:
                        try:
                            # a 태그에서 href 속성으로 단지번호 추출
                            link_element = item.find_element(By.CSS_SELECTOR, "a.inner")
                            href = link_element.get_attribute("href")
                            
                            # href에서 단지번호 추출 (/complex/info/22746 형식)
                            complex_id = ""
                            id_match = re.search(r"/complex/info/(\d+)", href)
                            if id_match:
                                complex_id = id_match.group(1)
                            
                            # 단지명 추출
                            name_element = item.find_element(By.CSS_SELECTOR, "span.keyword")
                            complex_name = name_element.text.strip()
                            
                            # 주소 추출
                            address_element = item.find_element(By.CSS_SELECTOR, "span.address")
                            address = address_element.text.strip()
                            
                            self.log(f"단지 발견: {complex_name} (위치: {address}, ID: {complex_id})")
                            
                            # 단지 정보 추가
                            complex_list.append({
                                "name": complex_name,
                                "address": address,
                                "id": complex_id
                            })
                            
                        except Exception as e:
                            self.log(f"검색 결과 항목 처리 중 오류: {str(e)}")
                            continue
                    
                    # 검색 결과가 여러 개면 선택 팝업 표시
                    if len(complex_list) > 1:
                        # 브라우저 종료
                        self.driver.quit()
                        self.driver = None
                        
                        # 선택 팝업 표시
                        dialog = ComplexSelectionDialog(self.root, "단지 선택", complex_list)
                        selected_complex = dialog.result
                        
                        if selected_complex and selected_complex["id"]:
                            # ID가 있는 경우 바로 매물 정보 수집
                            self.complex_data = selected_complex
                            self.log(f"단지번호 {selected_complex['id']}를 사용하여 매물 정보 수집을 시작합니다.")
                            self.download_data(selected_complex['id'], selected_complex['name'])
                        else:
                            self.log("단지 선택이 취소되었습니다.")
                        return
                    
                    elif len(complex_list) == 1 and complex_list[0]["id"]:
                        # 단일 결과이고 ID가 있는 경우
                        complex_data = complex_list[0]
                        
                        # 브라우저 종료
                        self.driver.quit()
                        self.driver = None
                        
                        self.complex_data = complex_data
                        self.log(f"단지번호 {complex_data['id']}를 사용하여 매물 정보 수집을 시작합니다.")
                        self.download_data(complex_data['id'], complex_data['name'])
                        return
                
                # 검색 결과 목록이 없는 경우 (단일 단지 페이지로 이동한 경우)
                current_url = self.driver.current_url
                self.log(f"현재 URL: {current_url}")
                
                # URL에서 단지 ID 추출
                pattern1 = r"complexes/(\d+)"
                pattern2 = r"complexNumber=(\d+)"
                pattern3 = r"/complex/info/(\d+)"
                
                match1 = re.search(pattern1, current_url)
                match2 = re.search(pattern2, current_url)
                match3 = re.search(pattern3, current_url)
                
                if match1:
                    complex_id = match1.group(1)
                elif match2:
                    complex_id = match2.group(1)
                elif match3:
                    complex_id = match3.group(1)
                else:
                    complex_id = None
                
                if complex_id:
                    # 단지명 추출 시도
                    try:
                        complex_name_element = self.driver.find_element(By.CSS_SELECTOR, ".complex_title, h2.text_title, .complex_name")
                        complex_name = complex_name_element.text.strip()
                    except:
                        complex_name = search_keyword
                    
                    self.log(f"URL에서 단지번호를 찾았습니다: {complex_id}")
                    
                    # 단지 정보 저장
                    self.complex_data = {
                        'name': complex_name,
                        'id': complex_id
                    }
                    
                    # 브라우저 종료
                    self.driver.quit()
                    self.driver = None
                    
                    # 매물 정보 수집 시작
                    self.download_data(complex_id, complex_name)
                    return
                        
            except Exception as e:
                self.log(f"검색 결과 처리 중 오류: {str(e)}")
                
                # 단일 단지 페이지로 이동한 경우 URL에서 ID 추출
                if not complex_list:
                    current_url = self.driver.current_url
                    self.log(f"단일 단지 페이지로 이동됨: {current_url}")
                    
                    # 단지명 추출
                    try:
                        complex_name = self.driver.find_element(By.CSS_SELECTOR, ".complex_title, h2.text_title").text
                    except:
                        complex_name = search_keyword
                    
                    # URL에서 단지 ID 추출
                    pattern1 = r"complexes/(\d+)"
                    pattern2 = r"complexNumber=(\d+)"
                    
                    match1 = re.search(pattern1, current_url)
                    match2 = re.search(pattern2, current_url)
                    
                    if match1:
                        complex_id = match1.group(1)
                        self.log(f"URL에서 단지번호를 찾았습니다: {complex_id}")
                        
                        # 단지 정보 저장
                        self.complex_data = {
                            'name': complex_name,
                            'id': complex_id
                        }
                        
                        # 브라우저 종료
                        self.driver.quit()
                        self.driver = None
                        
                        # 매물 정보 수집 시작
                        self.download_data(complex_id, complex_name)
                        return
                    elif match2:
                        complex_id = match2.group(1)
                        self.log(f"URL에서 단지번호를 찾았습니다: {complex_id}")
                        
                        # 단지 정보 저장
                        self.complex_data = {
                            'name': complex_name,
                            'id': complex_id
                        }
                        
                        # 브라우저 종료
                        self.driver.quit()
                        self.driver = None
                        
                        # 매물 정보 수집 시작
                        self.download_data(complex_id, complex_name)
                        return
            except Exception as e:
                self.log(f"검색 결과 처리 중 오류: {str(e)}")
            
            # 브라우저 종료
            self.driver.quit()
            self.driver = None
            
            # 검색 결과가 있으면 팝업창 표시
            if complex_list:
                # 검색 결과가 하나만 있고 ID가 있는 경우 바로 처리
                if len(complex_list) == 1 and complex_list[0]["id"]:
                    complex_data = complex_list[0]
                    self.complex_data = complex_data
                    self.log(f"단지번호 {complex_data['id']}를 사용하여 매물 정보 수집을 시작합니다.")
                    self.download_data(complex_data['id'], complex_data['name'])
                    return
                
                # 다중 검색 결과인 경우 선택 팝업창 표시
                dialog = ComplexSelectionDialog(self.root, "단지 선택", complex_list)
                selected_complex = dialog.result
                
                if selected_complex and selected_complex["id"]:
                    # ID가 있는 경우 바로 매물 정보 수집
                    self.complex_data = selected_complex
                    self.log(f"단지번호 {selected_complex['id']}를 사용하여 매물 정보 수집을 시작합니다.")
                    self.download_data(selected_complex['id'], selected_complex['name'])
                elif selected_complex:
                    # ID가 없는 경우 클릭하여 상세 페이지 접속 후 ID 추출
                    self.log(f"'{selected_complex['name']}' 단지를 선택했습니다. 단지번호를 찾는 중...")
                    self.search_complex_detail(selected_complex["name"])
                else:
                    self.log("단지 선택이 취소되었습니다.")
            else:
                self.log("검색 결과가 없습니다. 다른 검색어를 입력해보세요.")
                messagebox.showwarning("경고", "검색 결과가 없습니다. 다른 검색어를 입력해보세요.")
        
        except Exception as e:
            self.log(f"오류 발생: {str(e)}")
            messagebox.showerror("오류", f"검색 중 오류가 발생했습니다.\n{str(e)}")
            if self.driver:
                self.driver.quit()
                self.driver = None
        
        finally:
            self.search_button.config(state=tk.NORMAL)

    def search_complex_detail(self, complex_name):
     
        """선택한 단지의 상세 정보를 검색"""
        try:
            # Chrome 옵션 설정
            chrome_options = Options()
            chrome_options.add_argument("--headless")
            chrome_options.add_argument("--window-size=1920,1080")
            chrome_options.add_argument("--disable-gpu")
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")
            chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36")
            
            self.log("웹 브라우저를 초기화하는 중...")
            self.driver = webdriver.Chrome(options=chrome_options)
            
            # 네이버 부동산 접속
            self.log("네이버 부동산에 접속 중...")
            self.driver.get("https://fin.land.naver.com/")
            time.sleep(2)
            
            # 검색 버튼 클릭
            self.log("검색 버튼 클릭 중...")
            search_button = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "svg[viewBox='0 0 24 24']"))
            )
            search_button.click()
            time.sleep(1)
            
            # 검색창에 단지명 입력
            self.log(f"검색창에 '{complex_name}' 입력 중...")
            search_input = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.ID, "query"))
            )
            search_input.clear()
            search_input.send_keys(complex_name)
            search_input.send_keys(Keys.ENTER)
            time.sleep(3)
            
            # 검색 결과 목록에서 일치하는 단지 찾기
            try:
                # 검색 결과 목록 찾기
                complex_items = self.driver.find_elements(By.CSS_SELECTOR, "#complex_list_ul .result_item")
                
                if complex_items and len(complex_items) > 0:
                    # 첫 번째 결과 클릭
                    first_item = complex_items[0]
                    link = first_item.find_element(By.CSS_SELECTOR, "a.inner")
                    
                    self.log(f"검색 결과 클릭: {link.text}")
                    link.click()
                    time.sleep(3)
                    
                    # 현재 URL에서 단지번호 추출
                    current_url = self.driver.current_url
                    self.log(f"단지 상세 페이지 URL: {current_url}")
                    
                    # URL에서 단지 ID 추출
                    pattern1 = r"complexes/(\d+)"
                    pattern2 = r"complexNumber=(\d+)"
                    pattern3 = r"/complex/info/(\d+)"
                    
                    match1 = re.search(pattern1, current_url)
                    match2 = re.search(pattern2, current_url)
                    match3 = re.search(pattern3, current_url)
                    
                    complex_id = None
                    if match1:
                        complex_id = match1.group(1)
                    elif match2:
                        complex_id = match2.group(1)
                    elif match3:
                        complex_id = match3.group(1)
                    
                    if complex_id:
                        self.log(f"URL에서 단지번호를 찾았습니다: {complex_id}")
                        
                        # 단지 정보 저장
                        self.complex_data = {
                            'name': complex_name,
                            'id': complex_id
                        }
                        
                        # 브라우저 종료
                        self.driver.quit()
                        self.driver = None
                        
                        # 매물 정보 수집 시작
                        self.download_data(complex_id, complex_name)
                        return
                else:
                    self.log("검색 결과 목록을 찾을 수 없습니다.")
            except Exception as e:
                self.log(f"검색 결과 클릭 중 오류: {str(e)}")
            
            # 브라우저 종료
            if self.driver:
                self.driver.quit()
                self.driver = None
            
            self.log("단지번호를 찾을 수 없습니다.")
            messagebox.showwarning("경고", "단지번호를 찾을 수 없습니다. 다른 검색어를 입력해보세요.")
        
        except Exception as e:
            self.log(f"상세 검색 중 오류 발생: {str(e)}")
            messagebox.showerror("오류", f"상세 검색 중 오류가 발생했습니다.\n{str(e)}")
            if self.driver:
                self.driver.quit()
                self.driver = None
            
            self.log("단지번호를 찾을 수 없습니다.")
            messagebox.showwarning("경고", "단지번호를 찾을 수 없습니다. 다른 검색어를 입력해보세요.")
        
        except Exception as e:
            self.log(f"상세 검색 중 오류 발생: {str(e)}")
            messagebox.showerror("오류", f"상세 검색 중 오류가 발생했습니다.\n{str(e)}")
            if self.driver:
                self.driver.quit()
                self.driver = None

    def fetch_complex_info(self, complex_number, complex_name):
        """단지 상세 정보 수집"""
        try:
            self.log(f"단지 상세 정보를 수집하는 중...")
            
            # Chrome 옵션 설정
            chrome_options = Options()
            chrome_options.add_argument("--headless")
            chrome_options.add_argument("--window-size=1920,1080")
            chrome_options.add_argument("--disable-gpu")
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")
            chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36")
            
            # 웹드라이버 초기화
            service = Service(ChromeDriverManager().install())
            driver = webdriver.Chrome(service=service, options=chrome_options)
            
            # 새로운 네이버 부동산 단지 페이지 접속
            url = f"https://new.land.naver.com/complexes/{complex_number}"
            self.log(f"단지 페이지 접속: {url}")
            driver.get(url)
            time.sleep(5)  # 페이지 로딩 시간
            
            # 단지정보 버튼 클릭
            try:
                complex_info_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "button.complex_link"))
                )
                self.log("단지정보 버튼 찾음, 클릭 시도...")
                complex_info_button.click()
                time.sleep(3)  # 정보 로딩 기다림
            except Exception as e:
                self.log(f"단지정보 버튼 클릭 중 오류: {str(e)}")
            
            # 수집할 정보 초기화
            complex_info = {
                '단지명': complex_name,
                '단지번호': complex_number,
                '세대수': '',
                '저/최고층': '',
                '사용승인일': '',
                '총주차대수': '',
                '용적률': '',
                '건폐율': '',
                '건설사': '',
                '난방': '',
                '관리사무소': '',
                '주소': '',
                '면적': '',
                # 학교 정보 필드 추가
                '초등학교명': '',
                '배정동': '',
                '학교까지 거리': '',
                '학교 주소': '',
                '학교 전화': '',
                '설립 정보': '',
                '교육청': '',
                '교원수': '',
                '학생수': '',
                '학교 홈페이지': ''
            }
            
            # 정보 테이블에서 데이터 추출
            try:
                info_table = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "table.info_table_wrap"))
                )
                self.log("단지 정보 테이블 찾음")
                
                # 테이블의 모든 행 찾기
                rows = info_table.find_elements(By.CSS_SELECTOR, "tr.info_table_item")
                self.log(f"정보 테이블에서 {len(rows)}개 행 찾음")
                
                # 각 행에서 정보 추출
                for row in rows:
                    th_elements = row.find_elements(By.CSS_SELECTOR, "th.table_th")
                    td_elements = row.find_elements(By.CSS_SELECTOR, "td.table_td")
                    
                    # 한 행에 여러 TH-TD 쌍이 있을 수 있음
                    for i in range(len(th_elements)):
                        if i < len(td_elements):
                            label = th_elements[i].text.strip()
                            value = td_elements[i].text.strip()
                            
                            # 주소인 경우 모든 주소 정보 가져오기
                            if label == '주소':
                                address_elements = td_elements[i].find_elements(By.CSS_SELECTOR, "p.address")
                                if address_elements:
                                    value = " / ".join([addr.text.strip() for addr in address_elements])
                            
                            # 정보 저장
                            complex_info[label] = value
                            self.log(f"수집된 정보: {label} = {value}")
                
                # 결과 로깅
                info_count = sum(1 for v in complex_info.values() if v)
                self.log(f"단지 기본 정보 {info_count}개 항목 수집됨")
                
            except Exception as e:
                self.log(f"정보 테이블 처리 중 오류: {str(e)}")
                import traceback
                self.log(traceback.format_exc())
            
            # 학군정보 탭으로 이동
            try:
                # 학군정보 탭 찾기
                school_tabs = driver.find_elements(By.CSS_SELECTOR, "span.text")
                for tab in school_tabs:
                    if "학군정보" in tab.text:
                        self.log("학군정보 탭 찾음, 클릭 시도...")
                        tab.click()
                        time.sleep(3)  # 정보 로딩 기다림
                        break
                        
                # 학교 정보 가져오기
                school_detail = driver.find_elements(By.CSS_SELECTOR, "div.detail_box--school")
                if school_detail:
                    self.log("초등학교 정보 발견")
                    
                    # 학교명 추출
                    try:
                        school_name = school_detail[0].find_element(By.CSS_SELECTOR, "h5.heading_text").text.strip()
                        complex_info['초등학교명'] = school_name
                        self.log(f"초등학교명: {school_name}")
                    except Exception as e:
                        self.log(f"학교명 추출 오류: {str(e)}")
                    
                    # 배정동, 거리 정보 추출
                    try:
                        town_boxes = school_detail[0].find_elements(By.CSS_SELECTOR, "div.town_box")
                        for box in town_boxes:
                            title = box.find_element(By.CSS_SELECTOR, "div.town_title").text.strip()
                            detail = box.find_element(By.CSS_SELECTOR, "div.town_detail").text.strip()
                            
                            if "배정동" in title:
                                complex_info['배정동'] = detail
                                self.log(f"배정동: {detail}")
                            elif "학교까지" in title:
                                complex_info['학교까지 거리'] = detail
                                self.log(f"학교까지 거리: {detail}")
                    except Exception as e:
                        self.log(f"배정동/거리 정보 추출 오류: {str(e)}")
                    
                    # 학교 세부 정보 테이블 추출
                    try:
                        school_table = school_detail[0].find_element(By.CSS_SELECTOR, "table.info_table_wrap")
                        school_rows = school_table.find_elements(By.CSS_SELECTOR, "tr.info_table_item")
                        
                        for row in school_rows:
                            th = row.find_element(By.CSS_SELECTOR, "th.table_th").text.strip()
                            td = row.find_element(By.CSS_SELECTOR, "td.table_td").text.strip()
                            
                            if th == '주소':
                                complex_info['학교 주소'] = td
                                self.log(f"학교 주소: {td}")
                            elif th == '전화':
                                complex_info['학교 전화'] = td
                                self.log(f"학교 전화: {td}")
                            elif th == '설립':
                                complex_info['설립 정보'] = td
                                self.log(f"설립 정보: {td}")
                            elif th == '교육청':
                                complex_info['교육청'] = td
                                self.log(f"교육청: {td}")
                            elif th == '교원수':
                                complex_info['교원수'] = td
                                self.log(f"교원수: {td}")
                            elif th == '학생수':
                                complex_info['학생수'] = td
                                self.log(f"학생수: {td}")
                            elif th == '홈페이지':
                                try:
                                    link = row.find_element(By.CSS_SELECTOR, "a.link").get_attribute("href")
                                    complex_info['학교 홈페이지'] = link
                                    self.log(f"학교 홈페이지: {link}")
                                except:
                                    complex_info['학교 홈페이지'] = td
                                    self.log(f"학교 홈페이지: {td}")
                    except Exception as e:
                        self.log(f"학교 세부 정보 추출 오류: {str(e)}")
                        import traceback
                        self.log(traceback.format_exc())
            except Exception as e:
                self.log(f"학군정보 탭 처리 중 오류: {str(e)}")
                import traceback
                self.log(traceback.format_exc())
                
            # 전체 페이지 스크린샷 촬영
            try:
                # 스크롤 크기에 맞게 창 크기 조정
                total_height = driver.execute_script("return document.body.scrollHeight")
                driver.set_window_size(1920, total_height)
                time.sleep(1)  # 크기 조정 후 대기
                
                # 스크린샷 경로 설정
                screenshots_dir = os.path.join(self.save_path, "screenshots")
                os.makedirs(screenshots_dir, exist_ok=True)
                screenshot_path = os.path.join(screenshots_dir, f'{complex_name}_{complex_number}_info.png')
                
                # 스크린샷 저장
                driver.save_screenshot(screenshot_path)
                self.log(f"단지 정보 페이지 스크린샷 저장: {screenshot_path}")
                
                # 스크린샷 경로 저장
                self.complex_screenshot_path = screenshot_path
            except Exception as e:
                self.log(f"스크린샷 저장 중 오류: {str(e)}")
                self.complex_screenshot_path = None
            
            # 단지 정보 저장 (클래스 변수로)
            self.complex_detail_info = complex_info
            self.log(f"complex_detail_info 변수에 {len(complex_info)} 항목의 데이터 저장 완료")
            
            # 브라우저 종료
            driver.quit()
            
        except Exception as e:
            self.log(f"단지 상세 정보 수집 중 오류 발생: {str(e)}")
            import traceback
            self.log(traceback.format_exc())
            self.complex_screenshot_path = None
    
    def download_data(self, complex_number, complex_name):
        """선택한 단지의 매물 정보 다운로드 실행"""
        self.log(f"'{complex_name}' 단지(번호: {complex_number})의 매물 정보 수집을 시작합니다.")
        
        # 단지 상세 정보 먼저 수집 (동기적으로 실행)
        self.fetch_complex_info(complex_number, complex_name)
        # # 프로그레스 바 초기화
        # self.setup_progress_bar()
        
        # 스레드 작업 함수
        def download_worker():
            try:
                headers = {
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
                }
                
                all_properties = []
                
                # 첫 페이지 요청
                # url = f"https://fin.land.naver.com/front-api/v1/complex/article/list?complexNumber={complex_number}&userChannelType=PC&page=0"
                url = f"https://fin.land.naver.com/front-api/v1/complex/article/list?complexNumber={complex_number}&dateDescending=false&userChannelType=PC&page=0"
            
                
                self.log("첫 번째 페이지 데이터 요청 중...")
                
                try:
                    response = requests.get(url, headers=headers)
                    
                    if response.status_code != 200:
                        self.log(f"API 요청 실패: 상태 코드 {response.status_code}")
                        self.root.after(0, lambda: self.search_button.config(state=tk.NORMAL))
                        return
                    
                    data = response.json()
                    
                    if 'result' not in data or 'list' not in data['result']:
                        self.log("데이터 구조가 예상과 다릅니다.")
                        self.root.after(0, lambda: self.search_button.config(state=tk.NORMAL))
                        return
                    
                    property_list = data['result']['list']
                    
                    if not property_list:
                        self.log("매물 정보가 없습니다.")
                        self.root.after(0, lambda: self.search_button.config(state=tk.NORMAL))
                        return
                    
                    # 첫 페이지 데이터 처리
                    self.log(f"페이지 0에서 {len(property_list)}개의 매물 정보를 찾았습니다.")
                    for item in property_list:
                        property_data = self.extract_property_data(item, 0)
                        all_properties.append(property_data)
                    
                    # 다음 페이지 여부 확인
                    has_next_page = data['result'].get('hasNextPage', False)
                    
                    if has_next_page:
                        # 스레드 수를 5로 고정 (self.thread_var.get() 대신 5 사용)
                        max_threads = 5
                        self.log(f"다음 페이지 존재. {max_threads}개의 스레드로 병렬 처리를 시작합니다.")
                                      
                        # 향상된 배치 처리 방식 적용
                        # 작은 배치로 나누어 실행하고, 빈 결과가 있으면 중단
                        batch_size = 5  # 한 번에 처리할 페이지 수
                        page = 1
                        stop_processing = False
                        
                        while not stop_processing:
                            batch_end = page + batch_size
                            self.log(f"페이지 {page}~{batch_end-1} 배치 처리 중...")
                            
                            # 배치 내 페이지 병렬 처리
                            with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
                                future_to_page = {executor.submit(self.fetch_page, complex_number, p, headers): p 
                                                 for p in range(page, batch_end)}
                                
                                # 배치 내 페이지 결과 확인
                                empty_pages = 0
                                processed_pages = []
                                
                                for future in concurrent.futures.as_completed(future_to_page):
                                    page_num = future_to_page[future]
                                    processed_pages.append(page_num)
                                    
                                    try:
                                        page_properties = future.result()
                                        
                                        if page_properties:
                                            self.log(f"페이지 {page_num}에서 {len(page_properties)}개의 매물 정보를 찾았습니다.")
                                            all_properties.extend(page_properties)
                                            self.update_progress(len(all_properties))
                                        else:
                                            self.log(f"페이지 {page_num}에서 데이터가 없습니다.")
                                    except Exception as exc:
                                        self.log(f"페이지 {page_num} 처리 중 오류: {str(exc)}")
                                        empty_pages += 1
                                
                                # 배치의 절반 이상이 빈 페이지면 중단
                                if empty_pages >= batch_size * 0.5:
                                    self.log(f"빈 페이지가 많아 처리를 중단합니다.")
                                    stop_processing = True
                                    break
                                
                                # 다음 배치 시작 페이지 설정
                                if processed_pages:
                                    page = max(processed_pages) + 1
                                else:
                                    page = batch_end
                    
                    self.log(f"총 {len(all_properties)}개의 매물 정보를 수집했습니다.")
                    # 평형별 정보 수집 (방 개수, 화장실 개수)
                    # 평형별 정보 수집 (방 개수, 화장실 개수)
                    self.pyeong_info = self.fetch_pyeong_info(complex_number)
                    
                    # 결과 처리
                    self.root.after(0, lambda: self.process_results(all_properties, complex_name))
              
                except Exception as e:
                    self.log(f"데이터 요청 중 오류: {str(e)}")
                
                # 결과 처리 (메인 스레드에서 안전하게 처리하기 위해 after 사용)
                self.root.after(0, lambda: self.process_results(all_properties, complex_name))
                
            except Exception as e:
                self.log(f"데이터 수집 중 오류 발생: {str(e)}")
                # 에러 메시지는 메인 스레드에서 표시
                self.root.after(0, lambda: messagebox.showerror("오류", f"데이터 수집 중 오류가 발생했습니다.\n{str(e)}"))
                
            finally:
                # 버튼 상태 복원은 메인 스레드에서 처리
                self.root.after(0, lambda: self.search_button.config(state=tk.NORMAL))
        
        # 작업을 별도 스레드로 실행
        worker_thread = threading.Thread(target=download_worker, daemon=True)
        worker_thread.start()


    def apply_number_format(self, sheet, money_columns):
        """금액 열에 숫자 형식(천 단위 구분 기호) 적용"""
        try:
            # 헤더 찾기
            header_row = sheet[1]
            headers = [cell.value for cell in header_row]
            
            # 각 금액 열에 숫자 형식 적용
            for col_name in money_columns:
                if col_name in headers:
                    col_idx = headers.index(col_name) + 1  # 1-based index
                    col_letter = openpyxl.utils.get_column_letter(col_idx)
                    
                    # 모든 데이터 행에 대해 서식 적용
                    for row in range(2, sheet.max_row + 1):
                        cell = sheet[f"{col_letter}{row}"]
                        if cell.value is not None and cell.value != '':
                            # 천 단위 구분 기호를 사용하는 숫자 형식 적용
                            cell.number_format = '#,##0'
            
            # 평형 및 면적 열에도 소수점 숫자 형식 적용
            area_columns = ['전용면적', '공급면적', '전용평형', '공급평형']
            for col_name in area_columns:
                if col_name in headers:
                    col_idx = headers.index(col_name) + 1
                    col_letter = openpyxl.utils.get_column_letter(col_idx)
                    
                    for row in range(2, sheet.max_row + 1):
                        cell = sheet[f"{col_letter}{row}"]
                        if cell.value is not None and cell.value != '':
                            # 소수점 2자리 숫자 형식 적용
                            cell.number_format = '0.00'
                    
        except Exception as e:
            self.log(f"숫자 형식 적용 중 오류: {str(e)}")
    
            
    def process_results(self, all_properties, complex_name):
        """수집된 데이터 처리 (메인 스레드에서 실행)"""
        try:
            if not all_properties:
                self.log("수집된 매물 정보가 없습니다.")
                messagebox.showinfo("알림", "수집된 매물 정보가 없습니다.")
                return
            
            # 데이터프레임 생성
            df = pd.DataFrame(all_properties)
            
            # 최소 프리미엄 값이 있는 경우 프리미엄에 대입
            condition = (df['최소프리미엄'].notna()) & (df['최소프리미엄'] > 0)
            df.loc[condition, '프리미엄'] = df.loc[condition, '최소프리미엄']
            
            # 박스그래프 옵션이 활성화되었는지 확인
            if self.box_plot_var.get() and self.area_entry.get().strip():
                try:
                    target_area = self.area_entry.get().strip()
                    # 전용면적이 숫자인지 확인
                    if target_area.replace('.', '', 1).isdigit():
                        # 소수점 무시하고 정수로 변환
                        target_area = int(float(target_area))
                        self.log(f"전용면적 {target_area}㎡에 대한 박스그래프를 생성합니다.")
                        # 박스그래프 생성
                        self.create_box_plot(df, target_area)
                    else:
                        self.log("전용면적은 숫자로 입력해주세요.")
                        messagebox.showwarning("경고", "전용면적은 숫자로 입력해주세요.")
                except Exception as e:
                    self.log(f"박스그래프 생성 오류: {str(e)}")
                    messagebox.showerror("오류", f"박스그래프 생성 중 오류가 발생했습니다.\n{str(e)}")
                        # 데이터프레임 내용 확인 로깅
            if '매물번호' in df.columns:
                self.log(f"매물번호 컬럼 확인: {len(df['매물번호'].dropna())}개의 유효한 값이 있습니다.")
                # 첫 3개 매물번호 예시 출력
                self.log(f"첫 3개 매물번호 예시: {df['매물번호'].head(3).tolist()}")
            else:
                self.log("매물번호 컬럼이 없습니다.")
                
            if '매물링크' in df.columns:
                self.log(f"매물링크 컬럼 확인: {len(df['매물링크'].dropna())}개의 유효한 값이 있습니다.")
                # 첫 3개 매물링크 예시 출력
                self.log(f"첫 3개 매물링크 예시: {df['매물링크'].head(3).tolist()}")
            else:
                self.log("매물링크 컬럼이 없습니다.")
            
            # 전용면적과 공급면적을 숫자로 확실하게 변환
            for col in ['전용면적', '공급면적']:
                df[col] = pd.to_numeric(df[col], errors='coerce')
            
            # 이 부분이 수정 시작 지점입니다 
            # 방/화장실 정보 초기화 - 나중에 add_structure_to_excel에서 처리
            df['구조'] = ''  # 빈 문자열로 초기화
    
            # 필요하다면 디버깅용으로 pyeong_info 확인
            if hasattr(self, 'pyeong_info') and self.pyeong_info:
                self.log("\n===== 저장된 평형 정보 =====")
                for idx, info in enumerate(self.pyeong_info):
                    self.log(f"평형 {idx+1}: "
                            f"전용면적={info.get('exclusiveArea')}㎡, "
                            f"공급면적={info.get('supplyArea')}㎡, "
                            f"방={info.get('roomCount')}개, "
                            f"화장실={info.get('bathRoomCount')}개")
            # 이 부분이 수정 끝 지점입니다
                    
            # 평형 계산 (3.3으로 나누기)
            df['전용평형'] = (df['전용면적'] / 3.3).round(2)
            df['공급평형'] = (df['공급면적'] / 3.3).round(2)
            
            # 열 순서 재배치 - 사용자 지정 순서로 정렬
            # 사용자가 요청한 열 순서
            custom_column_order = [
                '등록날짜', '거래유형', '단지명', '동', '층/전체층', '전용면적', '전용평형', 
                '공급면적', '공급평형', '구조', '매매가', '보증금', '월세', '프리미엄', 
                '가격변동상태', '타입구분', '방향', '매물특징', '매물번호', '매물링크', '중개사명', 'VR노출여부', 
                '중개사 수', '최소매매가', '최대매매가', '최소보증금', '최대보증금', 
                '최소월세', '최대월세', '최소프리미엄', '최대프리미엄', '페이지번호'
            ]
            
            # 사용자 지정 열 순서로 먼저 추가 (존재하는 열만)
            desired_columns = []
            for col in custom_column_order:
                if col in df.columns:
                    desired_columns.append(col)
            
            # 나머지 열들 추가 (사용자 지정 목록에 없는 열들)
            for col in df.columns:
                if col not in desired_columns:
                    desired_columns.append(col)
            
            # 최종 열 순서 적용
            df = df[desired_columns]
            
            # 거래유형 변환
            trade_type_map = {
                'A1': '매매',
                'B1': '전세',
                'B2': '월세',
                'B3': '단기임대'
            }
            df['거래유형'] = df['거래유형'].map(lambda x: trade_type_map.get(x, x))
            
            # 방향 변환
            direction_map = {
                'SS': '남향',
                'SE': '남동향',
                'SW': '남서향',
                'EE': '동향',
                'WW': '서향',
                'NN': '북향',
                'NE': '북동향',
                'NW': '북서향'
            }
            df['방향'] = df['방향'].map(lambda x: direction_map.get(x, x))
            
            # 가격변동상태 변환
            price_change_map = {
                0: '변동없음',
                1: '가격상승',
                -1: '가격하락'
            }
            df['가격변동상태'] = df['가격변동상태'].map(lambda x: price_change_map.get(x, '알수없음'))
            
            # 중요: 금액 데이터는 문자열로 변환하지 않고 숫자로 유지
            # 다만 원 단위를 만원 단위로 변환 (숫자 /10000)
            money_columns = ['보증금', '월세', '매매가', '프리미엄', '최소매매가', '최대매매가', 
                             '최소보증금', '최대보증금', '최소월세', '최대월세', '최소프리미엄', '최대프리미엄',
                             '매물최저가']
            
            for col in money_columns:
                if col in df.columns:
                    df[col] = df[col].apply(lambda x: x/10000 if pd.notnull(x) and x > 0 else None)
            
            # 매매가 기준 내림차순 정렬 (숫자 값으로 정렬)
            if '매매가' in df.columns:
                df = df.sort_values(by='매매가', ascending=False, na_position='last')
            
            # 날짜 형식화
            today = datetime.now().strftime('%Y%m%d')
            
            # 엑셀 파일로 저장
            excel_filename = os.path.join(self.save_path, f'{complex_name}_매물정보_{today}.xlsx')
            has_complex_detail = hasattr(self, 'complex_detail_info') and self.complex_detail_info
            # 여러 시트로 저장 (오픈파이썬 엔진 사용)
            with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
                # 전체 매물 시트
                df.to_excel(writer, sheet_name='전체매물', index=False)
                try:
                    self.apply_filter_to_sheet(writer.sheets['전체매물'], len(df.columns))
                    self.apply_number_format(writer.sheets['전체매물'], money_columns)
                except Exception as e:
                    self.log(f"전체매물 시트 필터 적용 중 오류: {str(e)}")
                
                # 매매 매물만 필터링
                try:
                    deal_properties = df[df['매매가'].notna() & (df['매매가'] > 0)]
                    if not deal_properties.empty:
                        deal_properties.to_excel(writer, sheet_name='매매매물', index=False)
                        self.apply_filter_to_sheet(writer.sheets['매매매물'], len(deal_properties.columns))
                        self.apply_number_format(writer.sheets['매매매물'], money_columns)
                except Exception as e:
                    self.log(f"매매매물 시트 처리 중 오류: {str(e)}")
                
                # 전세 매물만 필터링
                try:
                    full_deposit = df[(df['보증금'].notna() & (df['보증금'] > 0)) & 
                                     (df['월세'].isna() | (df['월세'] <= 0))]
                    if not full_deposit.empty:
                        full_deposit = full_deposit.sort_values(by='보증금', ascending=False)
                        full_deposit.to_excel(writer, sheet_name='전세매물', index=False)
                        self.apply_filter_to_sheet(writer.sheets['전세매물'], len(full_deposit.columns))
                        self.apply_number_format(writer.sheets['전세매물'], money_columns)
                except Exception as e:
                    self.log(f"전세매물 시트 처리 중 오류: {str(e)}")
                
                # 월세 매물만 필터링
                try:
                    monthly_rent = df[(df['월세'].notna()) & (df['월세'] > 0)]
                    if not monthly_rent.empty:
                        monthly_rent = monthly_rent.sort_values(by='보증금', ascending=False)
                        monthly_rent.to_excel(writer, sheet_name='월세매물', index=False)
                        self.apply_filter_to_sheet(writer.sheets['월세매물'], len(monthly_rent.columns))
                        self.apply_number_format(writer.sheets['월세매물'], money_columns)
                except Exception as e:
                    self.log(f"월세매물 시트 처리 중 오류: {str(e)}")
                
                # 중개사 수별 매물 필터링
                try:
                    multi_realtor = df[df['중개사 수'] > 1]
                    if not multi_realtor.empty:
                        multi_realtor.to_excel(writer, sheet_name='중복매물', index=False)
                        self.apply_filter_to_sheet(writer.sheets['중복매물'], len(multi_realtor.columns))
                        self.apply_number_format(writer.sheets['중복매물'], money_columns)
                except Exception as e:
                    self.log(f"중복매물 시트 처리 중 오류: {str(e)}")
    
    
                # 단지 상세 정보 시트 추가
                # 단지 상세 정보 시트 추가
                # 단지 상세 정보 시트 추가
                # 단지 상세 정보 시트 추가
                try:
                    # 단지 정보 존재 여부 명시적 확인
                    self.log("단지 정보 추가 시작")
                    has_complex_detail = hasattr(self, 'complex_detail_info') and self.complex_detail_info
                    
                    if has_complex_detail:
                        self.log(f"단지 정보 확인됨: {len(self.complex_detail_info)} 항목")
                        
                        # 단지 상세 정보를 데이터프레임으로 변환
                        complex_info_df = pd.DataFrame([self.complex_detail_info])
                        # 필요한 열만 선택하고 순서 재배열
                        # 필요한 열만 선택하고 순서 재배열
                        columns_order = [
                            '단지명', '단지번호', '주소', '사용승인일', '세대수', '저/최고층', 
                            '난방', '총주차대수', '용적률', '건폐율', '관리사무소', '건설사', '면적',
                            # 학교 정보 필드 추가
                            '초등학교명', '배정동', '학교까지 거리', '학교 주소', '학교 전화',
                            '설립 정보', '교육청', '교원수', '학생수', '학교 홈페이지'
                        ]
                        # 존재하는 열만 필터링
                        available_columns = [col for col in columns_order if col in complex_info_df.columns]
                        complex_info_df = complex_info_df[available_columns]
                        
                        # 전치 (transpose) - 열과 행을 바꿈 (더 보기 좋게)
                        complex_info_df_t = complex_info_df.T.reset_index()
                        complex_info_df_t.columns = ['항목', '내용']
                        
                        # 단지 정보 시트 저장
                        complex_info_df_t.to_excel(writer, sheet_name='단지정보', index=False)
                        
                        # 열 너비 조정
                        sheet = writer.sheets['단지정보']
                        sheet.column_dimensions['A'].width = 20
                        sheet.column_dimensions['B'].width = 80  # 긴 내용 (주소, 면적 등) 고려하여 너비 증가
                        
                        # 헤더 스타일 적용
                        for cell in sheet[1]:
                            cell.font = Font(bold=True, size=11)
                            cell.fill = PatternFill(start_color="E0E0E0", end_color="E0E0E0", fill_type="solid")
                            cell.alignment = Alignment(horizontal='center', vertical='center')
                        
                        # 스크린샷이 있는 경우 이미지 시트 추가
                        if hasattr(self, 'complex_screenshot_path') and self.complex_screenshot_path and os.path.exists(self.complex_screenshot_path):
                            # 새 시트 생성
                            screenshot_sheet = writer.book.create_sheet("단지정보 스크린샷")
                            
                            # 이미지 삽입을 위한 openpyxl 라이브러리 사용
                            from openpyxl.drawing.image import Image
                            
                            try:
                                # 이미지 생성 및 크기 조정
                                img = Image(self.complex_screenshot_path)
                                
                                # 이미지 크기 조정 (엑셀 시트에 맞게)
                                max_width = 800
                                max_height = 1000
                                width_ratio = max_width / img.width if img.width > max_width else 1
                                height_ratio = max_height / img.height if img.height > max_height else 1
                                ratio = min(width_ratio, height_ratio)
                                
                                if ratio < 1:
                                    img.width = int(img.width * ratio)
                                    img.height = int(img.height * ratio)
                                
                                # 이미지 삽입
                                screenshot_sheet.add_image(img, 'A1')
                                
                                # 셀 너비 설정
                                screenshot_sheet.column_dimensions['A'].width = 120  # 적절한 크기로 설정
                                
                                self.log("단지정보 스크린샷 시트 추가 완료")
                            except Exception as e:
                                self.log(f"이미지 삽입 중 오류: {str(e)}")
                                import traceback
                                self.log(traceback.format_exc())
                        
                        self.log("단지 상세 정보 시트 추가 완료")
                except Exception as e:
                    self.log(f"단지 정보 시트 처리 중 오류: {str(e)}")
                    import traceback
                    self.log(traceback.format_exc())
    
    
    
    
    
                    
                # 열 너비 자동 조정
                for sheet_name in writer.sheets:
                    sheet = writer.sheets[sheet_name]
                    self.adjust_column_width(sheet)
                    # 매물링크 열에 하이퍼링크 추가
                    self.apply_hyperlinks(sheet)
    
            self.log(f"엑셀 파일 '{excel_filename}' 생성 완료!")
            self.log(f"총 {len(df)}개의 매물 정보가 저장되었습니다.")
            
            # 단지 정보 시트 추가 여부 확인
            # 단지 정보 시트 추가 여부 확인
            try:
                # with 구문 대신 명시적으로 열고 닫기
                wb = openpyxl.load_workbook(excel_filename)
                sheet_names = wb.sheetnames
                
                if '단지정보' in sheet_names:
                    self.log("단지정보 시트가 성공적으로 추가되었습니다.")
                else:
                    self.log("단지정보 시트가 추가되지 않았습니다.")
                
                if '단지정보 스크린샷' in sheet_names:
                    self.log("단지정보 스크린샷 시트가 성공적으로 추가되었습니다.")
                
                wb.close()  # 워크북 명시적으로 닫기
            except Exception as e:
                self.log(f"엑셀 파일 확인 중 오류: {str(e)}")
    
    
                
                # 단지 정보 시트 추가 여부 확인
                try:
                    wb = openpyxl.load_workbook(excel_filename)
                    sheet_names = wb.sheetnames
                    
                    if '단지정보' in sheet_names:
                        self.log("단지정보 시트가 성공적으로 추가되었습니다.")
                    else:
                        self.log("단지정보 시트가 추가되지 않았습니다.")
                    
                    if '단지정보 스크린샷' in sheet_names:
                        self.log("단지정보 스크린샷 시트가 성공적으로 추가되었습니다.")
                    
                    wb.close()  # 워크북 명시적으로 닫기
                except Exception as e:
                    self.log(f"엑셀 파일 확인 중 오류: {str(e)}")
    
            # 이 부분이 수정 시작 지점입니다 (파일 생성 후)
            # 구조 정보 추가 (전용면적 기준 매칭)
            if hasattr(self, 'pyeong_info') and self.pyeong_info:
                self.add_structure_to_excel(excel_filename, df)
            # 이 부분이 수정 끝 지점입니다
            
            # 저장 완료 메시지 표시
            messagebox.showinfo("완료", f"'{complex_name}' 단지의 매물 정보가 '{excel_filename}' 파일로 저장되었습니다.")
        
        except Exception as e:
            import traceback
            self.log(f"결과 처리 중 오류: {str(e)}")
            self.log(traceback.format_exc())
            messagebox.showerror("오류", f"결과 처리 중 오류가 발생했습니다.\n{str(e)}")

            
    def apply_filter_to_sheet(self, sheet, columns_count):
        """시트의 첫 행에 필터 적용"""
        try:
            # 안전한 필터 범위 설정
            # 필터 범위를 안전하게 제한 (최대 26개 열까지만)
            max_col = min(columns_count, 26)
            filter_range = f"A1:{chr(64 + max_col)}1"
            
            # 필터 적용 전 로깅
            self.log(f"필터 범위: {filter_range}, 총 열 수: {columns_count}")
            
            # 첫 행에 필터 적용
            sheet.auto_filter.ref = filter_range
            
            header_font = Font(bold=True, size=11)
            header_fill = PatternFill(start_color="E0E0E0", end_color="E0E0E0", fill_type="solid")
            
            for cell in sheet[1]:
                cell.font = header_font
                cell.fill = header_fill
                cell.alignment = Alignment(horizontal='center', vertical='center')
        except Exception as e:
            self.log(f"필터 적용 중 오류: {str(e)}")

    def adjust_column_width(self, sheet):
        """열 너비 자동 조정"""
        try:
            for column in sheet.columns:
                max_length = 0
                column_letter = column[0].column_letter
                
                for cell in column:
                    if cell.value:
                        # 셀 값의 길이 계산
                        try:
                            cell_length = len(str(cell.value))
                            max_length = max(max_length, cell_length)
                        except:
                            pass
                
                # 최소 너비 설정
                max_length = max(max_length, 10)
                # 최대 너비 제한
                max_length = min(max_length, 50)
                
                # 열 너비 설정 (약간의 여유 공간 추가)
                sheet.column_dimensions[column_letter].width = max_length + 2
        except Exception as e:
            self.log(f"열 너비 조정 중 오류: {str(e)}")

    
    def apply_hyperlinks(self, sheet):
        """매물링크 열에 하이퍼링크 추가"""
        try:
            # 헤더 찾기
            header_row = sheet[1]
            headers = [cell.value for cell in header_row]
            
            # 매물링크 열 인덱스 찾기
            link_col_idx = None
            for idx, header in enumerate(headers):
                if header == '매물링크':
                    link_col_idx = idx + 1  # 1-based index
                    break
            
            if not link_col_idx:
                self.log(f"매물링크 열을 찾을 수 없습니다.")
                return
            
            # 각 셀에 하이퍼링크 적용
            from openpyxl.styles import Font
            
            # 하이퍼링크 스타일
            hyperlink_font = Font(color="0000FF", underline="single")
            
            for row in range(2, sheet.max_row + 1):
                cell = sheet.cell(row=row, column=link_col_idx)
                url = cell.value
                
                if url and 'https://' in url:
                    # 하이퍼링크 추가
                    cell.hyperlink = url
                    cell.font = hyperlink_font
                    # 표시 텍스트 변경
                    cell.value = "매물 링크"
            
            self.log(f"하이퍼링크 추가 완료: {sheet.max_row-1}개 링크")
        except Exception as e:
            self.log(f"하이퍼링크 추가 중 오류: {str(e)}")
    def fetch_pyeong_info(self, complex_number):
        """단지의 평형별 정보를 수집 (방 개수, 화장실 개수 등)"""
        self.log(f"단지 {complex_number}의 평형별 정보를 수집하는 중...")
        
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
        }
        
        pyeong_info = []
        pyeong_type = 1
        max_attempts = 20  # 최대 시도 횟수
        
        while pyeong_type <= max_attempts:
            url = f"https://fin.land.naver.com/front-api/v1/complex/pyeong?complexNumber={complex_number}&pyeongTypeNumber={pyeong_type}"
                    
            
            try:
                response = requests.get(url, headers=headers)
                
                if response.status_code != 200:
                    self.log(f"평형 정보 요청 실패: 상태 코드 {response.status_code}")
                    break
                
                data = response.json()
    
                # 데이터가 없거나 실패한 경우 중단
                if not data or not data.get('isSuccess', False) or not data.get('result'):
                    self.log(f"평형 타입 {pyeong_type}에 대한 데이터가 없습니다. 수집 완료.")
                    break
                    
                # 데이터가 없으면 중단
                if not data or 'result' not in data or not data['result']:
                    break
                
                pyeong_data = data['result']
                
                # 상세 정보 로깅 (전용면적 정보 포함)
                self.log(f"평형 타입 {pyeong_type} 정보 수집: " 
                       f"전용면적={pyeong_data.get('exclusiveArea')}㎡, "
                       f"공급면적={pyeong_data.get('supplyArea')}㎡, "
                       f"방={pyeong_data.get('roomCount')}개, "
                       f"화장실={pyeong_data.get('bathRoomCount')}개")
                
                # 중요: exclusiveArea 필드도 저장
                pyeong_info.append({
                    'exclusiveArea': pyeong_data.get('exclusiveArea'),  # 전용면적 추가
                    'supplyArea': pyeong_data.get('supplyArea'),
                    'roomCount': pyeong_data.get('roomCount'),
                    'bathRoomCount': pyeong_data.get('bathRoomCount')
                })
                
                pyeong_type += 1
                
            except Exception as e:
                self.log(f"평형 정보 수집 중 오류: {str(e)}")
                break
        
        self.log(f"총 {len(pyeong_info)}개의 평형 정보를 수집했습니다.")
        
        # 디버깅: 수집된 평형 정보 확인
        for idx, info in enumerate(pyeong_info):
            self.log(f"수집된 평형 {idx+1}: " 
                   f"전용면적={info.get('exclusiveArea')}㎡, "
                   f"공급면적={info.get('supplyArea')}㎡, "
                   f"방={info.get('roomCount')}개, "
                   f"화장실={info.get('bathRoomCount')}개")
                   
        return pyeong_info
        
    def add_structure_to_excel(self, excel_path, df):
        """엑셀 파일에 구조(방/화장실) 정보를 직접 추가 - 전용면적 기준 매칭"""
        workbook = None
        try:
            self.log(f"엑셀 파일에 구조 정보 추가 중...")
            
            # 평형 정보 검증
            if not hasattr(self, 'pyeong_info') or not self.pyeong_info:
                self.log("평형 정보가 없습니다. 구조 정보를 추가할 수 없습니다.")
                return
                        
            # 평형 정보에 전용면적이 있는지 확인
            has_exclusive_area = any(p.get('exclusiveArea') is not None for p in self.pyeong_info)
            if not has_exclusive_area:
                self.log("평형 정보에 전용면적 데이터가 없습니다. 구조 정보를 추가할 수 없습니다.")
                return
            
            # 엑셀 파일 열기
            workbook = openpyxl.load_workbook(excel_path)
            
            # 평형 정보 로깅
            self.log("\n===== 평형별 정보 (전용면적 기준) =====")
            for idx, info in enumerate(self.pyeong_info):
                exclusive = info.get('exclusiveArea', '없음')
                rooms = info.get('roomCount', '없음')
                baths = info.get('bathRoomCount', '없음')
                self.log(f"평형 {idx+1}: 전용면적={exclusive}㎡, 방={rooms}개, 화장실={baths}개")
            
            # 각 시트에 구조 정보 추가
            for sheet_name in workbook.sheetnames:
                self.log(f"\n{sheet_name} 시트 처리 중...")
                sheet = workbook[sheet_name]
                
                # 헤더 찾기
                header_row = sheet[1]
                headers = [cell.value for cell in header_row]
                
                # 전용면적 열 인덱스 찾기
                exclusive_area_col_idx = None
                for idx, header in enumerate(headers, 1):
                    if header == '전용면적':
                        exclusive_area_col_idx = idx
                        break
                
                if not exclusive_area_col_idx:
                    self.log(f"시트 {sheet_name}에서 전용면적 열을 찾을 수 없습니다.")
                    continue
                    
                # 구조 열 인덱스 찾기 (이미 있는지 확인)
                structure_col_idx = None
                for idx, header in enumerate(headers, 1):
                    if header == '구조':
                        structure_col_idx = idx
                        break
                
                # 구조 열이 없으면 추가 (전용면적 다음에)
                if not structure_col_idx:
                    self.log(f"구조 열 추가 중...")
                    structure_col_idx = exclusive_area_col_idx + 1
                    sheet.insert_cols(structure_col_idx)
                    
                    # 헤더 설정
                    structure_cell = sheet.cell(row=1, column=structure_col_idx)
                    structure_cell.value = '구조'
                    
                    # 헤더 스타일 적용
                    structure_cell.font = Font(bold=True, size=11)
                    structure_cell.fill = PatternFill(start_color="E0E0E0", end_color="E0E0E0", fill_type="solid")
                    structure_cell.alignment = Alignment(horizontal='center', vertical='center')
                else:
                    self.log(f"기존 구조 열 (열 {structure_col_idx}) 사용")
                
                # 매칭 결과 카운터 초기화
                match_count = 0
                total_rows = sheet.max_row - 1  # 헤더 제외
                
                # 각 행에 구조 정보 추가
                for row_idx in range(2, sheet.max_row + 1):
                    # 전용면적 셀 값 가져오기
                    exclusive_area_cell = sheet.cell(row=row_idx, column=exclusive_area_col_idx)
                    
                    # 전용면적 값이 있는지 확인
                    if exclusive_area_cell.value:
                        try:
                            # 문자열을 숫자로 변환 (쉼표, 공백, 기타 문자 제거)
                            exclusive_area_str = str(exclusive_area_cell.value)
                            # 숫자만 추출
                            exclusive_area_str = ''.join(c for c in exclusive_area_str if c.isdigit() or c == '.')
                            
                            if not exclusive_area_str:
                                continue
                                
                            exclusive_area = float(exclusive_area_str)
                            
                            # 가장 가까운 평형 찾기 (전용면적 기준)
                            closest_info = None
                            min_diff = float('inf')
                            
                            # 10행마다 로깅
                            if row_idx % 10 == 0 or row_idx == 2:
                                self.log(f"행 {row_idx} 처리 중: 전용면적 = {exclusive_area}㎡")
                            
                            # 각 평형 정보와 비교
                            for info in self.pyeong_info:
                                if info.get('exclusiveArea') is None:
                                    continue
                                    
                                try:
                                    # 전용면적 비교
                                    pyeong_area = float(info.get('exclusiveArea'))
                                    diff = abs(exclusive_area - pyeong_area)
                                    
                                    if diff < min_diff:
                                        min_diff = diff
                                        closest_info = info
                                except (ValueError, TypeError) as e:
                                    if row_idx % 10 == 0:
                                        self.log(f"  전용면적 변환 오류: {str(e)}")
                            
                            # 허용 오차를 5.0㎡로 늘림 (더 많은 매칭을 위해)
                            if closest_info and min_diff <= 5.0:
                                rooms = closest_info.get('roomCount', '')
                                baths = closest_info.get('bathRoomCount', '')
                                if rooms and baths:
                                    structure = f"{rooms}룸 {baths}욕실"
                                    sheet.cell(row=row_idx, column=structure_col_idx).value = structure
                                    match_count += 1
                                    
                                    # 첫 5개와 마지막 5개 매칭 결과만 로깅
                                    if match_count <= 5 or match_count > total_rows - 5:
                                        self.log(f"  매칭 성공: 전용면적 {exclusive_area}㎡ → {structure} (차이: {min_diff:.2f}㎡)")
                                else:
                                    if row_idx % 10 == 0:
                                        self.log(f"  방/화장실 정보 없음")
                            else:
                                if row_idx % 10 == 0:
                                    self.log(f"  매칭 실패: 최소 차이 {min_diff:.2f}㎡ (허용 오차 5.0㎡)")
                                
                        except Exception as e:
                            if row_idx % 10 == 0:
                                self.log(f"행 {row_idx} 처리 중 오류: {str(e)}")
                
                self.log(f"{sheet_name} 시트 구조 정보 추가 완료: {match_count}/{total_rows} 행 매칭됨 ({match_count/total_rows*100:.1f}%)")
            
            # 변경사항 저장
            # 변경사항 저장
            workbook.save(excel_path)
            self.log("엑셀 파일에 구조 정보 추가 완료")
            
        except Exception as e:
            # 상세한 오류 정보 출력
            import traceback
            self.log(f"구조 정보 추가 중 오류: {str(e)}")
            self.log(traceback.format_exc())
        finally:
            # 항상 워크북 닫기
            if workbook:
                try:
                    workbook.close()
                except:
                    pass


        
    def fetch_page(self, complex_number, page, headers):
        """페이지별 데이터 요청 (병렬 처리용)"""
        # url = f"https://fin.land.naver.com/front-api/v1/complex/article/list?complexNumber={complex_number}&userChannelType=PC&page={page}"
        url = f"https://fin.land.naver.com/front-api/v1/complex/article/list?complexNumber={complex_number}&dateDescending=false&userChannelType=PC&page={page}"
            
        
        try:
            response = requests.get(url, headers=headers)
            
            if response.status_code != 200:
                return None
            
            data = response.json()
            
            if 'result' in data and 'list' in data['result']:
                property_list = data['result']['list']
                
                if not property_list:
                    return None
                
                # 매물 정보 추출
                page_properties = []
                for item in property_list:
                    property_data = self.extract_property_data(item, page)
                    page_properties.append(property_data)
                
                return page_properties
        except Exception as e:
            self.log(f"페이지 {page} 요청 오류: {str(e)}")
            return None
        
        return None

        
    def check_required_packages(self):
        """필요한 패키지가 설치되어 있는지 확인"""
        try:
            import matplotlib
            import numpy
        except ImportError:
            self.log("필요한 패키지가 설치되어 있지 않습니다. matplotlib와 numpy가 필요합니다.")
            messagebox.showwarning("경고", "박스그래프 생성을 위해 matplotlib와 numpy 패키지가 필요합니다.\n\n"
                                  "pip install matplotlib numpy 명령으로 설치해주세요.")
            
    def extract_property_data(self, item, page):
        """매물 항목에서 필요한 정보 추출"""
        rep_info = item['representativeArticleInfo']

        # 첫 번째 항목에 대해서만 매물 데이터 구조를 로깅 (디버깅용)
        if page == 0 and 'articleNo' not in rep_info:
            self.log(f"매물번호 필드를 찾을 수 없습니다. 응답 구조: {list(rep_info.keys())}")
        
        # 매물번호(articleNo)를 다양한 가능성으로 찾기 시도
        article_number = ''
        if 'articleNo' in rep_info:
            article_number = rep_info['articleNo']
        elif 'articleNumber' in rep_info:
            article_number = rep_info['articleNumber']
        elif 'article_no' in rep_info:
            article_number = rep_info['article_no']
        elif 'articleId' in rep_info:
            article_number = rep_info['articleId']
            
        # 매물 링크 생성
        article_link = f"https://fin.land.naver.com/articles/{article_number}" if article_number else ''
        
        
        # 기존 매물 정보 추출 코드에 공급면적 추가
        property_data = {
            '단지명': rep_info.get('complexName', ''),
            '동': rep_info.get('dongName', ''),
            '등록날짜': rep_info['verificationInfo'].get('exposureStartDate', ''),  # 등록날짜 추가       
            '매물번호': article_number,  # 매물번호 추가
            '매물링크': article_link,  # 매물 링크 추가
            '거래유형': rep_info.get('tradeType', ''),
            '전용면적': rep_info['spaceInfo'].get('exclusiveSpace', ''),
            '공급면적': rep_info['spaceInfo'].get('supplySpace', ''),  # supplySpace 필드로 공급면적 추가
            '구조': '',  # '방개수'와 '화장실개수' 대신 '구조'라는 하나의 문자열 필드
            '타입구분': rep_info['spaceInfo'].get('nameType', ''),
            '층/전체층': rep_info['articleDetail'].get('floorInfo', ''),
            '방향': rep_info['articleDetail'].get('direction', ''),
            '매물특징': rep_info['articleDetail'].get('articleFeatureDescription', ''),
            '보증금': rep_info['priceInfo'].get('warrantyPrice', 0),
            '월세': rep_info['priceInfo'].get('rentPrice', 0),
            '매매가': rep_info['priceInfo'].get('dealPrice', 0),
            '프리미엄': rep_info['priceInfo'].get('premiumPrice', 0),
            '가격변동상태': rep_info['priceInfo'].get('priceChangeStatus', 0),
            '중개사명': rep_info['brokerInfo'].get('brokerageName', ''),
            'VR노출여부': rep_info['articleMediaDto'].get('isVrExposed', False) if rep_info.get('articleMediaDto') else False,
            '중개사 수': 0,
            '최소매매가': 0,
            '최대매매가': 0,
            '최소보증금': 0,
            '최대보증금': 0,
            '최소월세': 0,
            '최대월세': 0,
            '최소프리미엄': 0,
            '최대프리미엄': 0,
            '페이지번호': page
        }
        
        # 중복 매물 정보 처리
        if 'duplicatedArticlesInfo' in item and item['duplicatedArticlesInfo'] is not None:
            # 중개사 수 추가
            property_data['중개사 수'] = item['duplicatedArticlesInfo'].get('realtorCount', 0)
            
            # 중복 매물 가격 정보
            if 'representativePriceInfo' in item['duplicatedArticlesInfo']:
                price_info = item['duplicatedArticlesInfo']['representativePriceInfo']
                
                if 'dealPrice' in price_info:
                    property_data['최소매매가'] = price_info['dealPrice'].get('minPrice', 0)
                    property_data['최대매매가'] = price_info['dealPrice'].get('maxPrice', 0)
                
                if 'warrantyPrice' in price_info:
                    property_data['최소보증금'] = price_info['warrantyPrice'].get('minPrice', 0)
                    property_data['최대보증금'] = price_info['warrantyPrice'].get('maxPrice', 0)
                
                if 'rentPrice' in price_info:
                    property_data['최소월세'] = price_info['rentPrice'].get('minPrice', 0)
                    property_data['최대월세'] = price_info['rentPrice'].get('maxPrice', 0)
                
                # 프리미엄 최소/최대 가격 추가
                if 'premiumPrice' in price_info:
                    property_data['최소프리미엄'] = price_info['premiumPrice'].get('minPrice', 0)
                    property_data['최대프리미엄'] = price_info['premiumPrice'].get('maxPrice', 0)
        
        return property_data

    def setup_progress_bar(self):
        """프로그레스 바 초기화"""
        # # 프로그레스 바가 이미 존재하면 제거
        # if hasattr(self, 'progress_frame'):
        #     self.progress_frame.destroy()
        
        # # 프레임 생성
        # self.progress_frame = ttk.Frame(self.root)
        # self.progress_frame.pack(fill="x", padx=10, pady=5)
        
        # # 레이블
        # self.progress_label = ttk.Label(self.progress_frame, text="데이터 수집 중...")
        # self.progress_label.pack(anchor="w", pady=(0, 5))
        
        # # 프로그레스 바
        # self.progress_bar = ttk.Progressbar(self.progress_frame, orient="horizontal", mode="indeterminate", length=300)
        # self.progress_bar.pack(fill="x")
        # self.progress_bar.start(10)
        
        # # 항목 카운터 레이블
        # self.count_label = ttk.Label(self.progress_frame, text="0개 매물 수집됨")
        # self.count_label.pack(anchor="e", pady=(5, 0))
        
        # # UI 업데이트
        # self.root.update_idletasks()
        pass
    
    def update_progress(self, count):
        """프로그레스 상태 업데이트 (스레드 안전)"""
        # def _update_progress():
        #     if hasattr(self, 'count_label'):
        #         self.count_label.config(text=f"{count}개 매물 수집됨")
        #         self.root.update_idletasks()
        
        # # 메인 스레드에서 실행
        # if threading.current_thread() is threading.main_thread():
        #     _update_progress()
        # else:
        #     # 다른 스레드에서는 after 메서드를 사용
        #     self.root.after(0, _update_progress)
        # 로그에만 정보 출력
        self.log(f"현재까지 {count}개 매물 수집됨")
    
    def complete_progress(self):
        """프로그레스 바 완료 처리 (스레드 안전)"""
        # def _complete_progress():
        #     if hasattr(self, 'progress_bar'):
        #         self.progress_bar.stop()
        #         self.progress_bar.configure(mode="determinate", value=100)
        #         self.progress_label.config(text="데이터 수집 완료")
        #         self.root.update_idletasks()
        
        # # 메인 스레드에서 실행
        # if threading.current_thread() is threading.main_thread():
        #     _complete_progress()
        # else:
        #     # 다른 스레드에서는 after 메서드를 사용
        #     self.root.after(0, _complete_progress)
        pass

# 메인 함수
def main():
    # 만료일 체크 (2025년 8월 31일까지만 실행)
    import datetime
    
    # # 현재 날짜
    # current_date = datetime.datetime.now().date()
    
    # # 만료일 (2025년 8월 31일)
    # expiry_date = datetime.date(2025, 8, 31)
    
    # # 만료일 체크
    # if current_date > expiry_date:
    #     # 만료된 경우 암묵적으로 종료 (아무 메시지 없이)
    #     return
    
    # 아직 유효한 경우 프로그램 실행
    root = tk.Tk()
    app = NaverRealEstateApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()
    

'잠실엘스' 단지 검색을 시작합니다...
웹 브라우저를 초기화하는 중...
네이버 부동산에 접속 중...
검색 버튼 클릭 중...
오류 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x0x93fd33+62915]
	GetHandleVerifier [0x0x93fd74+62980]
	(No symbol) [0x0x773e13]
	(No symbol) [0x0x7bc89e]
	(No symbol) [0x0x7bcc3b]
	(No symbol) [0x0x804ec2]
	(No symbol) [0x0x7e1424]
	(No symbol) [0x0x8026ea]
	(No symbol) [0x0x7e11d6]
	(No symbol) [0x0x7b0833]
	(No symbol) [0x0x7b16a4]
	GetHandleVerifier [0x0xba8d23+2590131]
	GetHandleVerifier [0x0xba3f6a+2570234]
	GetHandleVerifier [0x0x9659ea+217722]
	GetHandleVerifier [0x0x956058+153832]
	GetHandleVerifier [0x0x95c4bd+179533]
	GetHandleVerifier [0x0x947738+94152]
	GetHandleVerifier [0x0x9478c2+94546]
	GetHandleVerifier [0x0x932bda+9322]
	BaseThreadInitThunk [0x0x75ff5d49+25]
	RtlInitializeExceptionChain [0x0x77afd2fb+107]
	RtlGetAppContainerNamedObjectPath [0x0x77afd281+561]

